In [2]:
import json
import pandas as pd
import requests
import collections

api_key = "****************************""


In [3]:

####순서 1
def channel_dict(fileName):
    csv_df = pd.read_csv(fileName)
    channel_dict = {}
    for i in range(len(csv_df['youtuberName'])):
        channel_dict[csv_df['youtuberName'][i]] = csv_df['youtuberId'][i]
    return channel_dict
fileName = 'addYoutuber.csv'


####순서 2
def get_ratings_name_date(videoId, api_key):
    
    api_data = requests.get(f"https://www.googleapis.com/youtube/v3/videos?part=statistics&id={videoId}&key={api_key}")
    json_data = json.loads(api_data.content.decode("utf-8"))
    
    api_data1 = requests.get(f"https://www.googleapis.com/youtube/v3/videos?id={videoId}&key={api_key}&part=snippet")
    json_data1 = json.loads(api_data1.content.decode("utf-8"))
    
    try:
        ratings_name_date_dict = {"title": json_data1['items'][0]['snippet']['title'],
                     "publishedAt": json_data1['items'][0]['snippet']['publishedAt'][0:10],
                     "viewCount": json_data['items'][0]['statistics']['viewCount'],
                     "likeCount": json_data['items'][0]['statistics']['likeCount'],
                     "dislikeCount": json_data['items'][0]['statistics']['dislikeCount'],
                     "commentCount": json_data['items'][0]['statistics']['commentCount']}
        return ratings_name_date_dict
    except:
        pass

####순서 3
####이 함수 정의하고, 이 함수 통해 모든 videoInfo를 갖고온다!
def channel_with_video(channel_id, api_key):
    
    api_data = requests.get(f"https://www.googleapis.com/youtube/v3/channels?part=contentDetails&id={channel_id}&maxResults=50&key={api_key}")
    json_data = json.loads(api_data.content.decode("utf-8"))
    
    play_lists = []
    for item in range(0,len(json_data['items'])):
        play_lists.append(json_data['items'][item]['contentDetails']['relatedPlaylists']['uploads'])
        
    video_list = []
    for play_list_id in play_lists:
        api_data = requests.get(f"https://www.googleapis.com/youtube/v3/playlistItems?part=snippet&playlistId={play_list_id}&maxResults=50&key={api_key}")
        json_data = json.loads(api_data.content.decode("utf-8"))
      
        for item in range(len(json_data['items'])):
            if json_data['items'][item]['snippet']['publishedAt'][:4] == '2019': #여기에 날짜 조건 추가, 지금은 2019, 나중에 변경 가능
                video_list.append(json_data['items'][item]['snippet']['resourceId']['videoId'])
        nextPageToken = json_data.get("nextPageToken")
        print(f"{len(video_list)} videos")
        i = 1
        
        while nextPageToken and i < 4:
            api_data = requests.get(f"https://www.googleapis.com/youtube/v3/playlistItems?part=snippet&playlistId={play_list_id}&pageToken={nextPageToken}&maxResults=50&key={api_key}")
            json_data = json.loads(api_data.content.decode("utf-8"))

            for item in range(len(json_data['items'])):
                if json_data['items'][item]['snippet']['publishedAt'][:4] == '2019': #여기에 날짜 조건 추가, 지금은 2019, 나중에 변경 가능
                    video_list.append(json_data['items'][item]['snippet']['resourceId']['videoId'])
                else:
                    nextPageToken = False
            
            if nextPageToken == False:
                print("{0}: {1}개 비디오 수집 완료".format(channel_id, len(video_list)))
                
            elif i == 3:
                print("{0}: {1}개 비디오 수집 완료".format(channel_id, len(video_list)))
                break
            else:
                nextPageToken = json_data.get("nextPageToken")
                i += 1
                print(f"{len(video_list)} videos")
                
    
                
    videoInfo = {}
    print("It takes some time..")
    for Id in video_list:
        videoInfo[Id] = get_ratings_name_date(Id, api_key)
        
    channel_with_video = {channel_id: videoInfo}
    print("{0} channel all done!!".format(channel_id))
    
    return channel_with_video

def get_subscribe(channelId, api_key):
    api_data = requests.get(f"https://www.googleapis.com/youtube/v3/channels?part=statistics&id={channelId}&key={api_key}")
    json_data = json.loads(api_data.content.decode("utf-8"))
    global data
    try:
        data = json_data["items"][0]["statistics"]["subscriberCount"]
    except:
        pass
    return data

def change(name):
    name = name.replace(",", "")
    name = name.replace(" ", "_")
    name = name.replace("!", "")
    name = name.replace(":","")
    name = name.replace("[","")
    name = name.replace("]","")
    name = name.replace("/","")
    name = name.replace("&","")
    name = name.replace("(","")
    name = name.replace(")","")
    name = name.replace("'","")
    return name

In [6]:
df = pd.read_csv('youtuber.csv', encoding = 'utf-8')
channel_dict = {}
for i in range(len(df)):
    channel_dict[df['유튜버아이디'][i]] = df['유튜브링크'][i]

In [7]:
channel_video = []
for Id in list(channel_dict.values())[:10]:
    channel_video.append(channel_with_video(Id, api_key))

50 videos
UCW-rU_ZE4dAKDejWdTAKmoA: 54개 비디오 수집 완료
It takes some time..
UCW-rU_ZE4dAKDejWdTAKmoA channel all done!!
35 videos
UCsU2RlGgybcLzfmBqnTx-Rw: 35개 비디오 수집 완료
It takes some time..
UCsU2RlGgybcLzfmBqnTx-Rw channel all done!!
30 videos
UCaPQb6zHlBXzdMW_QzKh1Kw: 30개 비디오 수집 완료
It takes some time..
UCaPQb6zHlBXzdMW_QzKh1Kw channel all done!!
29 videos
UCSn90DAxCF3yRs6mFK0-8TA: 29개 비디오 수집 완료
It takes some time..
UCSn90DAxCF3yRs6mFK0-8TA channel all done!!
38 videos
UClSFlUAMQ-Yv002g8LlCqgQ: 38개 비디오 수집 완료
It takes some time..
UClSFlUAMQ-Yv002g8LlCqgQ channel all done!!
46 videos
UCWuWpKhbGev8Za4J-jfdSXw: 46개 비디오 수집 완료
It takes some time..
UCWuWpKhbGev8Za4J-jfdSXw channel all done!!
16 videos
UCZe_uzZ6VLMXtwW2V9-VvpQ: 16개 비디오 수집 완료
It takes some time..
UCZe_uzZ6VLMXtwW2V9-VvpQ channel all done!!
50 videos
UCuD_5vico-nuukZPXGrNYlQ: 67개 비디오 수집 완료
It takes some time..
UCuD_5vico-nuukZPXGrNYlQ channel all done!!
39 videos
UCENNpEHHCiZGb2dk1MHml7Q: 39개 비디오 수집 완료
It takes some time..
UCENNpEHH

In [11]:
subs = []
channelId = []
videoCount = []
likeCount = []
dislikeCount = []
viewCount = []
commentCount = []
recentFrequency = []

for i in range(len(channel_video)):
    for Id, videos in channel_video[i].items():
        subs.append(get_subscribe(Id, api_key))
        channelId.append(Id)    
        like = 0
        dislike = 0
        view = 0
        comment = 0
        frequency = 0
        v = 0
        for v_Id, Info in videos.items():
            try:
                like += int(Info['likeCount'])
                dislike += int(Info['dislikeCount'])
                view += int(Info['viewCount'])
                comment += int(Info['commentCount'])
                if Info['publishedAt'][5:7] == '06' or Info['publishedAt'][5:7] == '07':
                    frequency += 1
                v += 1
            except TypeError as e:
                print(e)
                print(v_Id)
                continue
        videoCount.append(v)
        likeCount.append(like)
        dislikeCount.append(dislike)
        viewCount.append(view)
        commentCount.append(comment)
        recentFrequency.append(round(frequency/6,2))
           

df_list = [channelId, videoCount, likeCount, dislikeCount, viewCount, commentCount, recentFrequency]

nameList = []
for name in list(channel_dict.keys())[:len(channel_video)]:
    nameList.append(name)
    
nameList = [change(name) for name in nameList]
    
data_df = pd.DataFrame()
data_df['channelName'] = nameList
data_df['subscriberCount'] = subs
data_df['videoCount'] = df_list[1]
data_df['likeCount'] = df_list[2]
data_df['dislikeCount'] = df_list[3]
data_df['viewCount'] = df_list[4]
data_df['commentCount'] = df_list[5]

'NoneType' object is not subscriptable
rsp_n2ZBzMY
'NoneType' object is not subscriptable
yHaYd8hqzHc
'NoneType' object is not subscriptable
1tNgF7old8o
'NoneType' object is not subscriptable
LL1dxm4REvE
'NoneType' object is not subscriptable
xy07Y9ST-Tc
'NoneType' object is not subscriptable
Nmyw1Fqh_LY
'NoneType' object is not subscriptable
kS_ONtThnXY
'NoneType' object is not subscriptable
zP0yf8P9_9Q
'NoneType' object is not subscriptable
Lw_vGw6tm2w
'NoneType' object is not subscriptable
t6ACDMvx0A0
'NoneType' object is not subscriptable
c6-EAJDSr1s
'NoneType' object is not subscriptable
TvOWkhE3lfw
'NoneType' object is not subscriptable
4d4uaGCczDo
'NoneType' object is not subscriptable
gY0UR61h-1k
'NoneType' object is not subscriptable
SXvwmlaqcEA
'NoneType' object is not subscriptable
f83IhOLKzdA
'NoneType' object is not subscriptable
J2epNx29y2E
'NoneType' object is not subscriptable
QF0jXqyjBu8
'NoneType' object is not subscriptable
PPLqxO4m7as
'NoneType' object is not subscr

In [12]:
data_df.head(10)

,channelName,subscriberCount,videoCount,likeCount,dislikeCount,viewCount,commentCount
0,Ha_Neul오늘의_하늘,810090,54,256100,6841,15486550,29028
1,여정을떠난여정,723281,33,138084,1418,5560246,16751
2,안구정화TV_Your_Korean_Friend,702370,29,809178,9979,13183901,33266
3,ondo온도,653228,17,272716,3780,9969269,18853
4,소진소진,562042,37,472228,20536,35927999,64376
5,영기TV,532009,41,135455,3313,9226747,12966
6,Sophie_Kim_하은,542542,16,199967,7113,2446235,39099
7,혜서니,517876,67,131777,6103,9449987,15815
8,sojoong쏘야쭝아,519119,37,251813,2704,10999027,14447
9,샒의_삶Serims_life,496140,59,302587,6006,17379967,23301
